# 1. Librerie

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import cvzone
import time


2023-06-19 21:20:51.337827: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# 2. Keypoints using MP Holistic

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model (to make our detection)
mp_drawing = mp.solutions.drawing_utils # Drawing utilities (to draw them)

In [4]:
def mediapipe_detection(image, model):

    """
    Esegue la rilevazione dei landmark delle mani utilizzando il modello di MediaPipe.

    Input:
    - image: l'immagine di input (frame) su cui effettuare la rilevazione dei landmark
    - model: il modello di MediaPipe per la rilevazione dei landmark

    Output:
    - image: l'immagine di input modificata, con i landmark disegnati
    - results: gli oggetti 'results' contenenti i risultati della rilevazione dei landmark
    """

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
def draw_landmarks(image, results):
    """
    Disegna i landmark delle mani sull'immagine utilizzando i risultati ottenuti dalla rilevazione di MediaPipe.

    Input:
    - image: l'immagine su cui disegnare i landmark
    - results: i risultati ottenuti dalla rilevazione di MediaPipe

    Output:
    - None
    """

    # Disegna i landmark della mano sinistra e le relative connessioni
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

    # Disegna i landmark della mano destra e le relative connessioni
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [6]:
def draw_styled_landmarks(image, results):
    """
    Disegna i landmark delle mani sull'immagine utilizzando uno stile personalizzato.

    Input:
    - image: l'immagine su cui disegnare i landmark
    - results: i risultati ottenuti dalla rilevazione di MediaPipe

    Output:
    - None
    """

    # Disegna i landmark della mano sinistra e le relative connessioni con uno stile personalizzato
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )

    # Disegna i landmark della mano destra e le relative connessioni con uno stile personalizzato
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [7]:
def extract_keypoints(results):
    """
    Estrae i punti chiave dei landmark delle mani dai risultati di MediaPipe.

    Input:
    - results: i risultati ottenuti dalla rilevazione di MediaPipe

    Output:
    - keypoints: un array numpy contenente i punti chiave dei landmark delle mani
    """

    # Estrae i punti chiave dei landmark della mano sinistra se disponibili, altrimenti crea un array di zeri
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)

    # Estrae i punti chiave dei landmark della mano destra se disponibili, altrimenti crea un array di zeri
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

    # Concatena i punti chiave delle mani sinistra e destra in un unico array
    keypoints = np.concatenate([lh, rh])

    return keypoints

In [8]:
# Azioni da riconoscere
actions = np.array(['0','1', '2', '3','4','5','altro'])

# 3. Import Neural Network

In [9]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(126,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model.load_weights('modello/action.h5')

# 4. Test in Real Time

In [10]:
colors = [(245, 117, 16), (117, 245, 16), (16, 117, 245), (255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0)]

def prob_viz(res, actions, input_frame, colors):

    """
    La funzione visualizza una barra di probabilità colorata per ogni azione prevista.
    Input:
    - res: Array di probabilità delle azioni previste.
    - actions: Array delle etichette delle azioni.
    - input_frame: Frame di input su cui disegnare le barre di probabilità.
    - colors: Lista dei colori corrispondenti alle azioni.
    
    Output:
    - output_frame: Frame di output con le barre di probabilità disegnate.
    """

    # Creazione di una copia del frame di input
    output_frame = input_frame.copy()
    
    for num, prob in enumerate(res):
        # Disegno del rettangolo colorato proporzionale alla probabilità dell'azione
        cv2.rectangle(output_frame, (0, 60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        
        # Aggiunta del testo dell'azione corrispondente sopra il rettangolo
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    return output_frame


# GAME

In [11]:
# Funzione per caricare le immagini dei gesti
def load_gesture_images():
    """Questa funzione carica le immagini dei gesti da una directory specifica. Restituisce una lista di immagini dei gesti.

    Input:
    - Nessun input richiesto.

    Output:
    - Una lista di immagini dei gesti.
    """
    gesture_images = [] # Crea una lista vuota per le immagini dei gesti
    for i in range(6): # Itera da 0 a 5 (6 iterazioni)
        image_path = f"gesture_images/{i}.png" # Ottiene il percorso dell'immagine corrente
        image = cv2.imread(image_path,cv2.IMREAD_UNCHANGED) # Carica l'immagine utilizzando OpenCV
        gesture_images.append(image) # Aggiunge l'immagine alla lista delle immagini dei gesti
    return gesture_images

def show_message_on_image(image, message, color, position=None):
    """
    Questa funzione mostra un messaggio sull'immagine specificata.

    Input:
    - image: L'immagine su cui mostrare il messaggio.
    - message: Il messaggio da mostrare sull'immagine.
    - color: Il colore del testo.
    - position (opzionale): La posizione del testo sull'immagine. Se non specificato, il testo verrà centrato sull'immagine.

    Output:
    - Nessun output restituito. L'immagine viene modificata direttamente.

    """
    font_scale = 1
    font_thickness = 2
    font_face = cv2.FONT_HERSHEY_SIMPLEX

    # Calcola le dimensioni del testo
    (text_width, text_height), _ = cv2.getTextSize(message, font_face, font_scale, font_thickness)

    # Calcola la posizione del testo in base alla posizione specificata o al centro dell'immagine
    if position is None:
        position = ((image.shape[1] - text_width) // 2, (image.shape[0] + text_height) // 2)

    # Disegna il testo sull'immagine
    cv2.putText(image, message, position, font_face, font_scale, color, font_thickness, cv2.LINE_AA)



# Nuove variabili per la rilevazione
sequence = []  # Lista vuota per la sequenza dei gesti
sentence = []  # Lista vuota per la frase composta dai gesti
predictions = []  # Lista vuota per le predizioni
threshold = 0.5  # Soglia di confidenza per considerare una predizione corretta

cap = cv2.VideoCapture(0)  # Inizializza la cattura video dalla webcam
cap.set(3, 640)  # Imposta la larghezza del frame a 640 pixel
cap.set(4, 480)  # Imposta l'altezza del frame a 480 pixel

timer = 0  # Variabile per il timer
stateResult = False  # Stato del risultato (se è stato mostrato un risultato)

gesture_images = load_gesture_images()  # Carica le immagini dei gesti
num_gestures = len(gesture_images)  # Numero totale di gesti
current_gesture = 0  # Indice del gesto corrente
prev_gesture = None  # Indice del gesto precedente
exit_flag = False  # Flag di uscita
game_started = False  # Flag per indicare se il gioco è iniziato
indovinato = False  # Flag per indicare se il gesto è stato indovinato

# Imposta il modello di MediaPipe per la rilevazione dei landmark
with mp.solutions.holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened() and not exit_flag:
        imgBG = cv2.imread("BG.png")  # Carica lo sfondo del gioco

        # Mostra il messaggio "Premi 'S' per iniziare" finché il gioco non è iniziato
        while not game_started:
            ret, frame = cap.read()  # Leggi il frame dalla webcam
            imgScaled = cv2.resize(frame, (0, 0), None, 0.875, 0.875)  # Ridimensiona il frame
            imgScaled = imgScaled[:, 80:480]  # Ritaglia il frame
            show_message_on_image(imgScaled, "Press 'S' to start", (255, 255, 255), position=(10, 30))  # Mostra messaggio personalizzato
            show_message_on_image(imgScaled, "START!", (0, 255, 0))  # Mostra messaggio centrato
            imgBG[234:654, 795:1195] = imgScaled  # Sovrapponi il frame allo sfondo
            cv2.imshow("BG", imgBG)  # Mostra l'immagine con lo sfondo

            key = cv2.waitKey(1)
            if key == ord('s'):
                game_started = True
                initialTime = time.time()

        ret, frame = cap.read()  # Leggi il frame dalla webcam
        imgScaled = cv2.resize(frame, (0, 0), None, 0.875, 0.875)  # Ridimensiona il frame
        imgScaled = imgScaled[:, 80:480]  # Ritaglia il frame

        # Effettua le rilevazioni dei landmark utilizzando il modello di MediaPipe
        image, results = mediapipe_detection(imgScaled, holistic)
        print(results)

        # Disegna i landmark stilizzati sull'immagine
        draw_styled_landmarks(image, results)

        # Logica di predizione
        keypoints = extract_keypoints(results)  # Estrai i landmark chiave

        res = model.predict(np.expand_dims(keypoints, axis=0))[0]  # Effettua la predizione utilizzando il modello
        print(actions[np.argmax(res)])  # Stampa l'azione corrispondente alla predizione
        predictions.append(np.argmax(res))  # Aggiungi la predizione alla lista delle predizioni

        if stateResult is False:
            # Visualizza l'immagine del gesto corrente DA RICONOSCERE
            if current_gesture < len(gesture_images):
                gesture_image = gesture_images[current_gesture]
                imgBG = cvzone.overlayPNG(imgBG, gesture_image, (149, 310))

            # Avvia un timer di 3 secondi
            timer = time.time() - initialTime
            cv2.putText(imgBG, str(int(timer)), (605, 435), cv2.FONT_HERSHEY_PLAIN, 6, (255, 0, 255), 4)

            if timer > 3:
                stateResult = True
                timer = 0

                # Logica di visualizzazione
                if np.unique(predictions[-10:])[0] == np.argmax(res) and res[np.argmax(res)] > threshold:
                    if len(sentence) > 0:
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

                if len(sentence) > 5:
                    sentence = sentence[-5:]

                # Verifica se il gesto corrente è stato indovinato
                if len(sentence) > 0 and str(current_gesture) == sentence[-1]:
                    prev_gesture = str(current_gesture)
                    current_gesture += 1
                    sentence = []
                    indovinato = True

                    # Mostra la scritta "OTTIMO LAVORO"
                    show_message_on_image(imgBG, "GOOD JOB", (0, 255, 0))
                    cv2.waitKey(500)

                    # Controlla se è stato raggiunto l'ultimo gesto
                    if current_gesture >= num_gestures:
                        current_gesture = 0  # Ripristina l'indice del gesto
                else:
                    # Mostra la scritta "RIPROVA"
                    show_message_on_image(imgBG, "TRY AGAIN", (0, 0, 255))
                    cv2.waitKey(500)

        imgBG[234:654, 795:1195] = image  # Sovrapponi l'immagine dei landmark allo sfondo

        if stateResult:
            imgBG = cvzone.overlayPNG(imgBG, gesture_image, (149, 310))  # Sovrapponi l'immagine del gesto corrente

        cv2.imshow("BG", imgBG)  # Mostra l'immagine con lo sfondo

        key = cv2.waitKey(1)
        if key == ord('s') or indovinato == True:
            game_started = True
            initialTime = time.time()
            stateResult = False
            indovinato = False

        # Interrompi in modo corretto
        if cv2.waitKey(1) & 0xFF == ord('q'):
            initialTime = time.time()
            exit_flag = True  # Imposta il flag di uscita

    cap.release()  # Rilascia la cattura video dalla webcam
    cv2.destroyAllWindows()  # Chiudi tutte le finestre
    cv2.waitKey(1)


2023-06-19 21:20:55.748 python[6616:349800] mac-virtualcam(DAL): PlugInMain version=1.3.0
2023-06-19 21:20:55.748 python[6616:349800] mac-virtualcam(DAL): HardwarePlugIn_QueryInterface 
2023-06-19 21:20:55.748 python[6616:349800] mac-virtualcam(DAL): HardwarePlugIn_Release sRefCount now = 0
2023-06-19 21:20:55.748 python[6616:349800] mac-virtualcam(DAL): HardwarePlugIn_InitializeWithObjectID self=0x138c40478
2023-06-19 21:20:55.749 python[6616:349800] mac-virtualcam(DAL): HardwarePlugIn_ObjectSetPropertyData OBSDALDevice(33) kCMIOObjectPropertyListenerAdded self=0x138c40478 data(int)=1684629094
2023-06-19 21:20:55.749 python[6616:349800] mac-virtualcam(DAL): HardwarePlugIn_ObjectSetPropertyData OBSDALDevice(33) kCMIOObjectPropertyListenerAdded self=0x138c40478 data(int)=1869180523
2023-06-19 21:20:55.749 python[6616:349800] mac-virtualcam(DAL): HardwarePlugIn_ObjectSetPropertyData OBSDALDevice(33) kCMIOObjectPropertyListenerAdded self=0x138c40478 data(int)=1885762592
2023-06-19 21:20:5

<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 83ms/step
altro
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 20ms/step
altro
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 19ms/step
altro
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 20ms/step
altro
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 20ms/step
altro
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 20ms/step
altro
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
altro
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
altro
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==================

2023-06-19 21:21:15.716 python[6616:355192] mac-virtualcam(DAL): PlugIn unhandled hasPropertyWithAddress for Unknown selector: ddsc


In [12]:
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

113